In [2]:
def bound_box(coord, w_factor = 1.5, h_factor = 1):
    [p1, p2, p3], p4 = coord, point_refer(coord)
    width = int_round((p4[0] - p1[0] + p3[0] - p2[0])/2.0)
    angle = rotation_angle([p4[0]-p1[0],p4[1]-p1[1]],[1,0])
    up_middle = int_round([ (p1[0]+ p4[0])/2.0 , (p1[1]+p4[1])/2.0])
#     w_factor = 2
#     h_factor = 1.5
    c_down_left, c_down_right = int_floor([up_middle[0] - width*w_factor, up_middle[1]]), \
                                 int_floor([up_middle[0]+width*w_factor, up_middle[1]])
    c_up_left, c_up_right = int_floor([c_down_left[0], c_down_left[1]-width*h_factor]),\
                             int_floor([c_down_right[0], c_down_right[1]-width*h_factor])

    bbox = [c_down_left, c_down_right, c_up_left, c_up_right ]

    return up_middle, angle, bbox

def point_refer(point_set):
    ''' in the x, y coordinate (ordinary one, not the image coornidate), 
        given the three points of a Parallelogram, which is positioned top-left, down-left, and down-right,
        based on these infos refer the forth one
    '''
    p1, p2, p3 = point_set
    if p2[1] > p1[1] and p2[0]< p3[0]: 
        return p1[0]-p2[0]+p3[0], p1[1]-p2[1]+p3[1]


def belong_to_line(line_p1, line_p2, p):
    ''' to decide whether a point is above or under a line'''

    # firstly, conform the above and the below position
    if line_p1[0] > line_p2[0]:
        p1,p2 = line_p1, line_p2
    elif line_p1[0] < line_p2[0]:
        p1, p2, = line_p2, line_p1
    else:
        p1, p2 = [line_p1, line_p2 ] if line_p1[1] > line_p2[1] else [line_p2, line_p1]

    
    v1 = complex( p1[0] - p2[0], p1[1] - p2[1] ) # the vecotor of the line
    v2 = complex( p[0] - p1[0], p[1] - p1[1] )  # the vector from p to input_p
    
    angle = rotation_angle(v2,v1)

    if angle == 0 or angle == 180:
        return 0
    # if the point is above the line
    elif 0 < angle < 180: 
        return 1
    # if the point is under the line
    else:
        return -1

    


def mask_rect(input_img, point_set):
    ''' the point set consist of three points and are the corner points of a rectangle in the input img,
        these points are the corner positioned top-left, down-left, and down-right. 
        Since the y-axis increases from top to down in the image coordinate, it must be transformed to the ordinal one.
    '''
    copy = as_image(input_img.copy())
    p1, p2, p3 = point_set
    p4 = point_refer(point_set)
    
    # transform the coordinate to the ordinal one
    p1, p2, p3, p4 = p2, p1, p4, p3
    if p1[0] != p2[0]:
        temp_a = -1 if (p1[0] > p2[0] and p1[1] > p2[1]) or (p1[0] < p2[0] and p2[1] >p1[1]) else 1
    else:
        temp_a = -1
    if p3[0] != p4[0]:
        temp_b = 1 if (p4[0] > p3[0] and p4[1] > p3[1]) or (p4[0] < p3[0] and p3[1] > p4[1]) else -1
    else:
        temp_b = 1
    
    out_rect = [[min(p1[0],p2[0]), max(p3[0], p4[0])], [min(p2[1], p3[1]), max(p1[1],p4[1])]]

    for x in range(out_rect[0][0],out_rect[0][1] +1):
        for y in range(out_rect[1][0], out_rect[1][1] +1): 
            if belong_to_line(p1,p2,[x,y]) in [0, temp_a] and belong_to_line(p3,p2,[x,y]) in [0, 1] \
                and belong_to_line(p3,p4,[x,y]) in [0, temp_b]  and belong_to_line(p1,p4,[x,y]) in [0, -1]:
                copy[y,x] = 255
                
    return copy